# Tugas 1
Zulvikar Harist

202110370311033

Model Scraping Data dan Extracting Data

In [ ]:
!pip install pandas requests beautifulsoup4 pdfminer.six lxml > /dev/null 2>&1

In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import argparse
import io
import os
import re
import time
import urllib
from concurrent.futures import ThreadPoolExecutor, wait
from datetime import date
import pandas as pd
import requests
from bs4 import BeautifulSoup
from pdfminer import high_level
from PyPDF2 import PdfReader

**Membuat Path**

In [ ]:
def create_path(folder_name):
    path = os.path.join(os.getcwd(), folder_name)
    if not os.path.exists(path):
        os.makedirs(path)
    return path

**Membuka Halaman Web**

In [ ]:
def open_page(link):
    count = 0
    while count < 3:
        try:
            return BeautifulSoup(requests.get(link).text, "lxml")
        except:
            count += 1
            time.sleep(5)

In [ ]:
# Extract detail from HTML
def get_detail(soup, keyword):
    try:
        text = (
            soup.find(lambda tag: tag.name == "td" and keyword in tag.text)
            .find_next()
            .get_text()
            .strip()
        )
        return text
    except:
        return ""

**Pengambilan informasi Barang Bukti dengan elemen "ul"**

In [ ]:
def get_barang_bukti(soup):
    try:
        ul_element = soup.find(lambda tag: tag.name == "ul")
        if ul_element:
            return " ".join([li.get_text(strip=True) for li in ul_element.find_all("li")])
        return ""
    except:
        return ""

**Download dan Membaca PDF**

In [ ]:
def get_pdf(url, path_pdf):
    file = urllib.request.urlopen(url)
    file_name = file.info().get_filename().replace("/", " ")
    file_content = file.read()
    with open(f"{path_pdf}/{file_name}", "wb") as out_file:
        out_file.write(file_content)
    return io.BytesIO(file_content), file_name

**PreProsessing Teks**

In [ ]:
def preprocess_text(text):
    """
    Membersihkan teks dari string yang tidak diinginkan.
    Berlaku untuk teks dari HTML atau PDF.
    """
    # String yang tidak diinginkan
    unwanted_strings = [
        "M a h ka m a h A g u n g R e p u blik In d o n esia\n",
        "Disclaimer\n",
        "Kepaniteraan Mahkamah Agung Republik Indonesia berusaha untuk selalu mencantumkan informasi paling kini dan akurat sebagai bentuk komitmen Mahkamah Agung untuk pelayanan publik, transparansi dan akuntabilitas\n",
        "pelaksanaan fungsi peradilan. Namun dalam hal-hal tertentu masih dimungkinkan terjadi permasalahan teknis terkait dengan akurasi dan keterkinian informasi yang kami sajikan, hal mana akan terus kami perbaiki dari waktu kewaktu.\n",
        "Dalam hal Anda menemukan inakurasi informasi yang termuat pada situs ini atau informasi yang seharusnya ada, namun belum tersedia, maka harap segera hubungi Kepaniteraan Mahkamah Agung RI melalui :\n",
        "Email : kepaniteraan@mahkamahagung.go.id    Telp : 021-384 3348 (ext.318)\n",
        "Direktori Putusan Mahkamah Agung Republik Indonesia",
        "putusan.mahkamahagung.go.id",
        "hkama ahkamah Agung Repub ahkamah Agung Republik Indonesia mah Agung Republik Indonesia blik Indonesi",
        "Disclaimer Kepaniteraan Mahkamah Agung Republik Indonesia berusaha untuk selalu mencantumkan informasi paling kini dan akurat sebagai bentuk komitmen Mahkamah Agung untuk pelayanan publik, transparansi dan akuntabilitas pelaksanaan fungsi peradilan. Namun dalam hal-hal tertentu masih dimungkinkan terjadi permasalahan teknis terkait dengan akurasi dan keterkinian informasi yang kami sajikan, hal mana akan terus kami perbaiki dari waktu kewaktu. Dalam hal Anda menemukan inakurasi informasi yang termuat pada situs ini atau informasi yang seharusnya ada, namun belum tersedia, maka harap segera hubungi Kepaniteraan Mahkamah Agung RI melalui : Email : kepaniteraan@mahkamahagung.go.id Telp : 021-384 3348 (ext.318)",
    ]

    # Hapus string yang tidak diinginkan
    for unwanted in unwanted_strings:
        text = text.replace(unwanted, "")

    # Hapus variasi kata "mengadili"
    text = re.sub(r"(?i)^[\sMENGADILI:]*m[\s]*e[\s]*n[\s]*g[\s]*a[\s]*d[\s]*i[\s]*l[\s]*i[\s:]*", "", text)

    # Tambahkan spasi jika teks tidak memiliki spasi
    if not re.search(r"\s", text):
        text = " ".join(re.findall(r"[A-Za-z]+", text))

    return text.strip()


**pengambilan dan menyimpan informasi data pada google drive**

In [ ]:
def extract_data(link, keyword_url):
    global today, path_output, path_pdf, path_putusan

    path_output = '/content/drive/MyDrive/UAS TKI/Data'
    path_pdf = '/content/drive/MyDrive/UAS TKI/Data/PDF'
    path_putusan = '/content/drive/MyDrive/UAS TKI/Data/TXT'
    today = date.today().strftime("%Y-%m-%d")

    os.makedirs(path_output, exist_ok=True)
    os.makedirs(path_pdf, exist_ok=True)
    os.makedirs(path_putusan, exist_ok=True)

    soup = open_page(link)
    table = soup.find("table", {"class": "table"})
    table.find("h2").decompose()

    nomor = preprocess_text(get_detail(table, "Nomor"))
    lembaga_peradilan = preprocess_text(get_detail(table, "Lembaga Peradilan"))
    barang_bukti = preprocess_text(get_barang_bukti(table))
    catatan_amar = preprocess_text(get_detail(table, "Catatan Amar"))

    try:
        link_pdf = soup.find("a", href=re.compile(r"/pdf/"))
        if link_pdf:
            file_pdf, file_name_pdf = get_pdf(link_pdf["href"], path_pdf)

            # Ekstraksi teks dari PDF
            reader = PdfReader(file_pdf)
            text_pdf = "".join([page.extract_text() for page in reader.pages])
            text_pdf = preprocess_text(text_pdf)

            # Simpan teks ke file TXT dalam folder putusan
            txt_file_path = os.path.join(path_putusan, f"{file_name_pdf}_{today}.txt")
            with open(txt_file_path, 'w', encoding='utf-8') as txt_file:
                txt_file.write(text_pdf)
        else:
            text_pdf, file_name_pdf = "", ""
    except Exception as e:
        print(f"Error processing PDF: {e}")
        link_pdf = ""
        text_pdf = ""
        file_name_pdf = ""

    # Validasi data sebelum ditambahkan ke DataFrame
    data = [
        nomor if nomor else "N/A",
        lembaga_peradilan if lembaga_peradilan else "N/A",
        barang_bukti if barang_bukti else "N/A",
        catatan_amar if catatan_amar else "N/A",
    ]

    # Filter baris yang kosong atau tidak valid
    if any(data):
        result = pd.DataFrame(
            [data],
            columns=[
                "Nomor",
                "Lembaga Peradilan",
                "Barang Bukti",
                "Catatan Amar",
            ],
        )

        keyword_url = keyword_url.replace("/", " ")
        if keyword_url.startswith("https"):
            keyword_url = ""

        destination = f"{path_output}/putusan_ma_PNTangerang.csv"

        # Simpan dengan penanganan file yang ada
        if not os.path.isfile(destination):
            result.to_csv(destination, mode="w", encoding="utf-8", index=False)
        else:
            result.to_csv(destination, mode="a", encoding="utf-8", header=False, index=False)

**Melakukan Crawling**

In [ ]:
# Run scraper
def run_scraper(keyword=None, url=None, sort_date=True, download_pdf=True):
    if not keyword and not url:
        print("Please provide a keyword or URL")
        return

    path_output = '/content/drive/MyDrive/Temu Kembali Informasi/Data/PDF'
    path_pdf = create_path("pdf-putusan")

    today = date.today().strftime("%Y-%m-%d")

    link = f"https://putusan3.mahkamahagung.go.id/search.html?q={keyword}&page=1"

    if url:
        link = url

    soup = open_page(link)

    last_page = int(
        soup.find_all("a", {"class": "page-link"})[-1].get("data-ci-pagination-page")
    )

    if url:
        print(f"Scraping with url: {url} - {1 * last_page} data - {last_page} page")
    else:
        print(f"Scraping with keyword: {keyword} - {1 * last_page} data - {last_page} page")

    if url:
        keyword_url = url
    else:
        keyword_url = keyword

    futures = []
    with ThreadPoolExecutor(max_workers=4) as executor:
        for page in range(last_page):
            futures.append(
                executor.submit(run_process, keyword_url, page + 1, sort_date)
            )
    wait(futures)

In [ ]:
def run_process(keyword_url, page, sort_page):

    if keyword_url.startswith("https"):
        link = f"{keyword_url}&page={page}"
    else:
        link = f"https://putusan3.mahkamahagung.go.id/search.html?q={keyword_url}&page={page}"
    if sort_page:
        link = f"{link}&obf=TANGGAL_PUTUS&obm=desc"

    print(link)

    soup = open_page(link)
    links = soup.find_all("a", {"href": re.compile("/direktori/putusan")})

    for link in links:
        extract_data(link["href"], keyword_url)

def scrape_specific_url(url, download_pdf=True):
    if not url or not url.startswith("https://"):
        print("Please provide a valid URL")
        return

    extract_data(url, url)

In [ ]:
def scrape_specific_url(url, download_pdf=True):
    if not url or not url.startswith("https://"):
        print("Please provide a valid URL")
        return

    extract_data(url, url)

**Crawling pada PN PalangkaRaya**

In [ ]:
# Download Pidana
run_scraper(url="https://putusan3.mahkamahagung.go.id/search.html?q=&jenis_doc=putusan&cat=3c40e48bbab311301a21c445b3c7fe57&jd=&tp=&court=097598PN66|097598PN66%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20&t_reg=&t_upl=&t_pr=&t_put=2024")

Scraping with url: https://putusan3.mahkamahagung.go.id/search.html?q=&jenis_doc=putusan&cat=3c40e48bbab311301a21c445b3c7fe57&jd=&tp=&court=097598PN66|097598PN66%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20&t_reg=&t_upl=&t_pr=&t_put=2024 - 15 data - 15 page
https://putusan3.mahkamahagung.go.id/search.html?q=&jenis_doc=putusan&cat=3c40e48bbab311301a21c445b3c7fe57&jd=&tp=&court=097598PN66|097598PN66%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20&t_reg=&t_upl=&t_pr=&t_put=2024&page=1&obf=TANGGAL_PUTUS&obm=desc
https://putusan3.mahkamahagung.go.id/search.html?q=&jenis_doc=putusan&cat=3c40e48bbab311301a21c445b3c7fe57&jd=&tp=&court=097598PN66|097598PN66%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20&t_reg=&t_upl=&t_pr=&t_put=2024&page=2&obf=TANGGAL_PUTUS&obm=desc
https://putusan3.mahkamahagung.go.id/search.html?q=&jenis_doc=putusan&cat=3c40e48bbab311301a21c445b3c7fe57&jd=&tp=&court=097598PN66|097598PN66%20%20%20%20%20%20%20%20%20%20%